## Startup Instructions ###

Must connect locally to Cloud SQL Auth proxy before this notebook can run. 
Instructions for downloading proxy found [here](https://cloud.google.com/sql/docs/postgres/quickstart-proxy-test#install-proxy)

Start the proxy in its own terminal so you can monitor its output. Replace <INSTANCE_CONNECTION_NAME> with the instance connection name you copied in the previous step. Replace <PORT> with '5432' for postgres, or '1433' postgres port already in use.
```
./cloud_sql_proxy -instances=<INSTANCE_CONNECTION_NAME>=tcp:<PORT>
```
Example for current database connection:
```
./cloud_sql_proxy -instances=plenary-era-308716:us-central1:denver-real-estate=tcp:1433 \
-credential_file=<PATH TO CREDENTIAL FILE> &
```
A message similar to the following appears:
```
Listening on 127.0.0.1:5432 for myproject:us-central1:myinstance".
Ready for new connections
```

In [1]:
import sqlalchemy, os
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Text, String, Boolean, Float, DateTime
from sqlalchemy.dialects.postgresql import JSON, JSONB
import pandas as pd

The following assumes 'SQL_CLOUD_USER' and 'SQL_CLOUD_PASS' are already saved as environmental variables. Please email me at Olivia.a.Carnes@gmail.com to request credentials.

In [2]:
user = os.environ['SQL_CLOUD_USER']
password = os.environ['SQL_CLOUD_PASS']

In [3]:
engine = sqlalchemy.create_engine("postgresql://{}:{}@127.0.0.1:1433/postgres".format(user, password))
connection = engine.connect()
metadata = sqlalchemy.MetaData()
metadata.reflect(bind=engine)

Currently available tables include 
* 'listings_query': all listings and some facts
* 'listings_detailed': all listings and detailed facts
* 'building_query': building info for apartments

In [4]:
df = pd.read_sql_table('listings_detailed', connection)

In [5]:
df.sort_values(by=['zpid'])

,zpid,DateAdded,DateModified,city,state,homeStatus,bedrooms,bathrooms,price,yearBuilt,...,monthlyHoaFee,parentRegion,propertyTaxRate,taxHistory,buildingId,daysOnZillow,isListedByOwner,pageViewCount,favoriteCount,isIncomeRestricted
9710,13292327,2021-04-09 21:02:25.269311,2021-04-09 21:02:25.269342,Denver,CO,RECENTLY_SOLD,4.0,3.0,597500,1943.0,...,NaN,{'name': 'Regis'},0.53,"{'0': {'time': 1586487744980, 'value': 41192, ...",None,239,False,17.0,0.0,None
9856,13292367,2021-04-09 21:10:28.188232,2021-04-09 21:10:28.188254,Denver,CO,RECENTLY_SOLD,3.0,1.0,410000,1920.0,...,NaN,{'name': 'Regis'},0.53,"{'0': {'time': 1586488228071, 'value': 25397, ...",None,339,False,12.0,0.0,None
9855,13292376,2021-04-09 21:10:26.768474,2021-04-09 21:10:26.768496,Denver,CO,RECENTLY_SOLD,2.0,1.0,375000,1906.0,...,NaN,{'name': 'Regis'},0.53,"{'0': {'time': 1586488226714, 'value': 30774, ...",None,337,False,21.0,0.0,None
10543,13292399,2021-04-09 21:50:08.986378,2021-04-09 21:50:08.986402,Denver,CO,SOLD,2.0,1.0,309900,1922.0,...,NaN,{'name': 'Regis'},0.53,"{'0': {'time': 1586490608787, 'value': 29151, ...",None,477,False,4.0,0.0,None
10672,13293802,2021-04-09 21:57:16.772498,2021-04-09 21:57:16.772520,Denver,CO,SOLD,2.0,1.0,360000,1942.0,...,NaN,{'name': 'Regis'},0.53,"{'0': {'time': 1586491036339, 'value': 26798, ...",None,932,False,1.0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146,2145249874,2021-04-09 12:50:28.784568,2021-04-09 12:50:28.784591,Denver,CO,RECENTLY_SOLD,2.0,3.0,630000,2000.0,...,NaN,{'name': 'Cherry Creek'},0.53,{},1001937453,66,False,26.0,0.0,None
611,2146117575,2021-04-09 11:10:11.591959,2021-04-09 11:10:11.591987,Denver,CO,RECENTLY_SOLD,2.0,2.0,501500,2007.0,...,NaN,{'name': 'Capitol Hill'},0.53,{},2073262420,28,False,20.0,0.0,None
5222,2146963759,2021-04-09 15:47:50.603961,2021-04-09 15:47:50.603983,Denver,CO,SOLD,2.0,1.0,415000,1954.0,...,NaN,{'name': 'Cheesman Park'},0.53,{},None,387,False,4.0,0.0,None
4862,2146969821,2021-04-09 15:26:39.767134,2021-04-09 15:26:39.767167,Denver,CO,SOLD,2.0,3.0,570000,2006.0,...,NaN,{'name': 'City Park West'},0.53,{},None,1092,False,0.0,0.0,None


In [6]:
df = pd.read_sql_table('listings_query', connection)

In [7]:
df['DateAdded'].max()

Timestamp('2021-04-13 17:36:13.162133')

In [21]:
df_detail = pd.read_sql_table('listings_detailed', connection)

In [22]:
df_building = pd.read_sql_table("building_query", connection)